# Overview

This notebook uses the 'TextFeaturesGenerator' class (from text_features) to convert textual data into qunatitaive data. 

For now, it creates a bag-of-words representation and a tf-idf representation. We will also add SVD/PCA of these matrices and a Word2Vec representation in the next few days.

Will update the TextFeaturesGenerator class on an ongoing basis and update the usage here.

In [1]:
import sys
sys.path.append('..') #to add top-level to path

from modules.text_features import TextFeaturesGenerator
from modules.project_helper import TweetData
import pandas as pd
import numpy as np
from datetime import timedelta  
import datetime

Reusing the TweetData class to get cleaned tweets.

In [2]:
tweet_data = TweetData()
tweet_data.clean_tweets.head()

,tweets,timestamp,after4_date
timestamp,,,
2019-11-17 19:57:12-06:00,tell jennifer williams whoever that is to read...,2019-11-17 19:57:12-06:00,2019-11-18
2019-11-17 19:56:02-06:00,,2019-11-17 19:56:02-06:00,2019-11-18
2019-11-17 19:49:47-06:00,paul krugman of has been wrong about me from t...,2019-11-17 19:49:47-06:00,2019-11-18
2019-11-17 19:47:32-06:00,schiff is a corrupt politician,2019-11-17 19:47:32-06:00,2019-11-18
2019-11-17 19:30:09-06:00,blew the nasty amp obnoxious chris wallace wil...,2019-11-17 19:30:09-06:00,2019-11-18


# Daily Tweets

This does the following two things:

1) Change the date of the tweets after 3 PM Chicago time to the following day (as trading closes then)
2) Concatenate all tweets in a given day to one large document

In [3]:
tweet_data.daily_tweets.head()

,tweets
date,
2009-05-05,donald trump will be appearing on the view tom...
2009-05-08,donald trump reads top ten financial tips on l...
2009-05-09,new blog post celebrity apprentice finale and ...
2009-05-12,my persona will never be that of a wallflower ...
2009-05-13,miss usa tara conner will not be fired ive alw...


# Feature Generator

Creating a 'TextFeaturesGenerator' instance which takes the tweets as an argument

In [4]:
feature_generator = TextFeaturesGenerator(tweet_data.clean_tweets.tweets)

'get_bow_matrix' creates the bag-of-words matrix

In [5]:
bow_mat = feature_generator.get_bow_matrix()

In [6]:
bow_mat.shape

(28813, 17035)

The shape of this matrix is 27.96K rows (same number as the tweets) and the columns are 16,781, which is equal to the unique number of words in the vocabulary.

In [7]:
bow_mat[:10,:10].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

As you can see, most of the values are zero which is why it is stored as a 'sparse-matrix'

Bag-of-words is simply a count of words in the tweet. A better representation is 'tf-idf'. The 'get_tfidf_matrix' creates

In [8]:
tfidf_mat = feature_generator.get_tfidf_matrix()
tfidf_mat.shape

(28813, 17035)

The matrices can be saved using the matrices function. You can either specify a 'folder' which will be created and both matrices stored in it, else will store in the working directory.

In [9]:
feature_generator.save_matrices()

The two matrices will be saved with the names "bow_mat.npz" and "tfidf_mat.npz"

You can also specify a folder and a suffix to the file names.

In [10]:
#feature_generator.save_matrices(folder="../data/intermediate_data/matrices/",suffix="_v2")

The files can be loaded using the following commands:

In [11]:
from scipy import sparse
bow_loaded = sparse.load_npz("bow_mat.npz")
tfidf_loaded = sparse.load_npz("tfidf_mat.npz")
print(bow_loaded.shape)
print(tfidf_loaded.shape)

(28813, 17622)
(28813, 17622)


## PCA (through SVD) of the matrices

You can get the SVD of the bow and tfidf matrices as well.

In [12]:
svd_bow_mat = feature_generator.get_svd_bow_mat()

In [13]:
svd_bow_mat.shape

(28813, 2)

By default, it gives back two components. You can changet that using the n_components argument.

In [14]:
svd_bow_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [15]:
svd_bow_mat.shape

(28813, 100)

You can get the SVD of the tf-idf as well.

In [16]:
svd_tfidf_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [17]:
svd_tfidf_mat.shape

(28813, 100)

These matrices can be saved as well.

In [18]:
feature_generator.save_matrices()

You can load them back using np.load

In [19]:
svd_loaded_mat = np.load('svd_tfidf_mat.npy')

In [20]:
svd_loaded_mat.shape

(28813, 100)

# Aggregagte SVD per day 

In [21]:
svd_df = pd.DataFrame(svd_loaded_mat)

In [22]:
svd_df['timestamp'] = tweet_data.clean_tweets.index
svd_df['date'] = svd_df.timestamp.dt.date

In [23]:
svd_df.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,timestamp,date
0,3.834487,1.049659,-0.752303,0.540785,0.697841,1.161297,-0.285807,0.096066,0.443987,0.020685,...,0.001997,0.283183,-0.000505,-0.116874,-0.648558,0.039167,0.324678,-0.332617,2019-11-17 19:57:12-06:00,2019-11-17
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019-11-17 19:56:02-06:00,2019-11-17
2,3.066449,0.750298,0.961831,-0.716980,1.161889,1.909849,0.004423,-0.184612,-0.648819,-0.123790,...,0.061167,0.216344,0.307724,-0.243496,-0.058589,0.098019,0.024762,-0.102572,2019-11-17 19:49:47-06:00,2019-11-17
3,0.200860,-0.106529,0.113348,0.882736,-0.032079,0.144637,-0.060565,0.017510,-0.003573,-0.142311,...,0.008687,0.018517,0.013596,0.003546,-0.025227,-0.008678,-0.004813,-0.034114,2019-11-17 19:47:32-06:00,2019-11-17
4,2.924057,0.136825,0.787196,-0.582478,1.228916,-0.823097,-0.837244,-0.844682,-0.611131,-0.136084,...,-0.030496,-0.322623,-0.153485,-0.305056,0.102308,0.079475,0.083862,0.094577,2019-11-17 19:30:09-06:00,2019-11-17


In [24]:
svd_df_daily = svd_df.groupby('date').agg(np.mean)

In [25]:
svd_df_daily

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
date,,,,,,,,,,,,,,,,,,,,,
2009-05-04,1.913749,-0.747906,-0.003787,-0.296547,0.094471,-0.787979,0.052940,-0.854261,-0.816232,-0.307401,...,-0.253444,0.008926,-0.099195,0.135415,-0.066571,-0.026697,0.103465,0.191053,-0.092000,-0.234838
2009-05-05,1.730476,-0.740193,-0.031689,-0.506165,-0.143328,-0.601006,-0.989515,-0.805250,-0.493852,-0.394849,...,-0.044059,-0.160107,-0.092636,-0.035800,0.054119,-0.006889,0.145136,-0.021547,-0.120819,-0.220128
2009-05-08,0.657294,0.015502,0.343751,-0.131198,-0.185005,-0.137012,-0.157285,-0.142959,-0.297246,-0.015065,...,-0.052403,-0.080637,-0.073000,0.040072,-0.009655,-0.071746,0.058368,-0.007542,0.054075,-0.034596
2009-05-12,0.758495,-0.617789,-0.254773,-0.132794,0.894611,-0.351745,-0.331128,-0.610386,0.315728,-0.345302,...,0.037456,-0.192383,0.184599,0.088568,-0.098376,-0.118087,0.051968,0.065329,-0.017629,0.089943
2009-05-13,0.549742,-0.715264,-0.649868,-0.032409,-0.050086,0.132020,0.063894,-0.103547,-0.264288,-0.087137,...,-0.038108,-0.189308,-0.114422,-0.101642,0.047657,-0.089017,0.117612,-0.059856,-0.057277,-0.218494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-13,2.519770,0.247129,-0.212118,0.924762,-0.322130,-0.118895,-0.063160,0.127476,0.160089,-0.188983,...,0.093924,0.040859,-0.043925,-0.018499,0.066512,-0.071368,0.140639,-0.066795,-0.138603,0.023189
2019-11-14,2.669994,0.579700,-0.108959,0.189455,-0.083211,0.040116,-0.114184,0.223222,0.147902,0.217054,...,0.137473,0.000656,-0.029687,-0.077313,-0.067358,-0.007301,0.066117,-0.059155,-0.035581,-0.026165
2019-11-15,1.964659,-0.180836,-0.009068,0.079289,0.116708,-0.011782,0.171617,0.010124,0.046587,0.054752,...,0.056781,0.045793,-0.080318,0.015343,-0.076486,-0.041644,0.083092,-0.036122,-0.090159,0.023869


In [26]:
svd_df_daily.to_csv('svd_df_daily.csv')

# 4 PM

In [27]:
tweet_data.clean_tweets['timestamp'] = tweet_data.clean_tweets.index
after_4_tweets = tweet_data.clean_tweets.timestamp.dt.hour >= 15
tweet_data.clean_tweets['after4_date'] = tweet_data.clean_tweets.timestamp.dt.date
tweet_data.clean_tweets.loc[after_4_tweets,'after4_date'] = tweet_data.clean_tweets.timestamp[after_4_tweets].dt.date + timedelta(days=1)

In [28]:
tweet_data.clean_tweets.head(100)

,tweets,timestamp,after4_date
timestamp,,,
2019-11-17 19:57:12-06:00,tell jennifer williams whoever that is to read...,2019-11-17 19:57:12-06:00,2019-11-18
2019-11-17 19:56:02-06:00,,2019-11-17 19:56:02-06:00,2019-11-18
2019-11-17 19:49:47-06:00,paul krugman of has been wrong about me from t...,2019-11-17 19:49:47-06:00,2019-11-18
2019-11-17 19:47:32-06:00,schiff is a corrupt politician,2019-11-17 19:47:32-06:00,2019-11-18
2019-11-17 19:30:09-06:00,blew the nasty amp obnoxious chris wallace wil...,2019-11-17 19:30:09-06:00,2019-11-18
...,...,...,...
2019-11-12 11:25:11-06:00,why is such a focus put on nd and rd hand witn...,2019-11-12 11:25:11-06:00,2019-11-12
2019-11-12 03:07:37-06:00,a great try by we are all proud of you,2019-11-12 03:07:37-06:00,2019-11-12
2019-11-12 01:33:57-06:00,vote for sean spicer on dancing with the stars...,2019-11-12 01:33:57-06:00,2019-11-12


In [29]:
combined_daily_tweets = tweet_data.clean_tweets.groupby('after4_date')['tweets'].apply(lambda x: ' '.join(x))
combined_daily_tweets.head()

after4_date
2009-05-05    donald trump will be appearing on the view tom...
2009-05-08    donald trump reads top ten financial tips on l...
2009-05-09    new blog post celebrity apprentice finale and ...
2009-05-12    my persona will never be that of a wallflower ...
2009-05-13    miss usa tara conner will not be fired ive alw...
Name: tweets, dtype: object

In [30]:
combined_daily_tweets.to_csv('../data/intermediate_data/combined_daily_tweets.csv')

/Users/lalopey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


# Check if the concatenation is correct

In [31]:
tweet_data.clean_tweets.tweets[tweet_data.clean_tweets.after4_date==pd.to_datetime("2019-10-03")]

timestamp
2019-10-03 13:40:19-05:00    fake news just like the snakes and gators in t...
2019-10-03 12:09:33-05:00      schiff is a lowlife who should resign at least 
2019-10-03 11:36:23-05:00    schiff is a lying disaster for our country he ...
2019-10-03 11:33:00-05:00     the republican party has never had such support 
2019-10-03 11:31:53-05:00    book is doing really well a study in unfairnes...
2019-10-03 11:29:53-05:00                                      thank you hugh 
2019-10-03 11:28:49-05:00       a great book by a brilliant author buy it now 
2019-10-03 11:22:55-05:00                                   great job richard 
2019-10-03 10:52:11-05:00                       keep up the great work kellie 
2019-10-03 10:37:33-05:00    the ukraine controversy continues this morning...
2019-10-03 10:00:00-05:00    the u s won a billion award from the world tra...
2019-10-02 23:41:51-05:00                democrats want to steal the election 
2019-10-02 23:27:52-05:00    mississippi t

In [32]:
combined_daily_tweets[combined_daily_tweets.index.values==pd.to_datetime("2019-10-03")]

after4_date
2019-10-03    fake news just like the snakes and gators in t...
Name: tweets, dtype: object

# Create SVD matrix of the combiened 4 PM tweets

In [33]:
combined_generator = TextFeaturesGenerator(combined_daily_tweets)

In [34]:
n_components = 2
combined_svd_df = pd.DataFrame(combined_generator.get_svd_tfidf_mat(n_components=n_components))

In [35]:
combined_svd_df['after4_date'] = combined_daily_tweets.index.values

In [36]:
combined_svd_df.head()

,0,1,after4_date
0,0.229813,0.195852,2009-05-05
1,0.052079,0.062505,2009-05-08
2,0.079677,0.035545,2009-05-09
3,0.097449,0.042823,2009-05-12
4,0.069333,0.061978,2009-05-13


In [37]:
combined_svd_df.to_csv('combined_svd_df.csv')

# Scoring Tweets

In [38]:
tweet_data = TweetData()
tweet_data.clean_tweets.head()

,tweets,timestamp,after4_date
timestamp,,,
2019-11-17 19:57:12-06:00,tell jennifer williams whoever that is to read...,2019-11-17 19:57:12-06:00,2019-11-18
2019-11-17 19:56:02-06:00,,2019-11-17 19:56:02-06:00,2019-11-18
2019-11-17 19:49:47-06:00,paul krugman of has been wrong about me from t...,2019-11-17 19:49:47-06:00,2019-11-18
2019-11-17 19:47:32-06:00,schiff is a corrupt politician,2019-11-17 19:47:32-06:00,2019-11-18
2019-11-17 19:30:09-06:00,blew the nasty amp obnoxious chris wallace wil...,2019-11-17 19:30:09-06:00,2019-11-18


In [39]:
tweet_data.daily_tweets.head()

,tweets
date,
2009-05-05,donald trump will be appearing on the view tom...
2009-05-08,donald trump reads top ten financial tips on l...
2009-05-09,new blog post celebrity apprentice finale and ...
2009-05-12,my persona will never be that of a wallflower ...
2009-05-13,miss usa tara conner will not be fired ive alw...


Split into train at test a certain date (in the example, 2018-01-01)

In [40]:
train_tweets = tweet_data.daily_tweets[tweet_data.daily_tweets.index<=pd.to_datetime("2018-01-01")]
score_tweets = tweet_data.daily_tweets[tweet_data.daily_tweets.index>pd.to_datetime("2018-01-01")]

Create the feature generator class

In [41]:
feature_generator_with_scores = TextFeaturesGenerator(train_tweets.tweets,score_tweets.tweets)

In [42]:
train_svd, test_svd = feature_generator_with_scores.get_svd_tfidf_mat(n_components=10)

In [43]:
print(train_svd.shape)
print(test_svd.shape)

(2395, 10)
(682, 10)


Convert to dataframe and add date

In [44]:
train_svd_df = pd.DataFrame(train_svd)
train_svd_df['date'] = train_tweets.index

train_svd_df = pd.DataFrame(train_svd)
train_svd_df['date'] = train_tweets.index
train_svd_df.head()

,0,1,2,3,4,5,6,7,8,9,date
0,0.255172,0.092125,0.183204,0.255826,0.088298,0.041543,-0.003421,-0.017150,0.035320,0.055567,2009-05-05
1,0.060732,0.019309,0.076875,0.052297,0.092406,0.027281,-0.026346,0.003407,0.051978,0.010131,2009-05-08
2,0.081247,0.017581,0.069671,0.133248,-0.036836,0.034915,0.136022,-0.045027,0.013207,-0.018155,2009-05-09
3,0.104220,0.007641,0.050820,0.006971,0.090410,0.004721,-0.042391,0.035917,0.017452,0.032376,2009-05-12
4,0.076733,0.024047,0.069054,0.042180,0.072445,0.012045,-0.011133,0.025644,-0.001294,0.033007,2009-05-13


In [45]:
test_svd_df = pd.DataFrame(test_svd)
test_svd_df['date'] = score_tweets.index
test_svd_df.head()

,0,1,2,3,4,5,6,7,8,9,date
0,0.475856,-0.057664,-0.083165,-0.011788,-0.023193,-0.024715,0.113311,0.070669,-0.003210,-0.047515,2018-01-02
1,0.480844,-0.088982,-0.084319,0.005735,-0.024957,-0.011321,-0.008410,0.069149,-0.024373,-0.034131,2018-01-03
2,0.395566,-0.069907,-0.077053,-0.018488,-0.022293,-0.041572,0.045141,0.032323,0.001137,-0.013918,2018-01-04
3,0.442383,-0.027245,-0.129938,0.006728,0.001319,-0.036676,0.073748,0.046921,0.063788,-0.054366,2018-01-05
4,0.365892,-0.071711,-0.074081,0.010866,-0.013835,0.084723,0.026269,0.051004,0.048330,-0.044919,2018-01-06


In [46]:
plt.plot(final_daily_tweets.groupby('after4_date').max().single_ret)

NameError: name 'plt' is not defined